## First, imports:

In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from astropy import units

In [ ]:
# Import my library:

import os
import sys

apt_path = os.path.abspath(os.path.join('..', 'apostletools'))
sys.path.append(apt_path)

import snapshot
import subhalo
import dataset_comp
import curve_fit

In [ ]:
import importlib
importlib.reload(snapshot)
importlib.reload(subhalo)
importlib.reload(dataset_comp)
importlib.reload(curve_fit)

# Counting Subhalo Bound Particles

## Motivation

I use $v_\mathrm{max} = \max_{r} \sqrt{\frac{G M(<r)}{r}}$ as a measure of the mass of a subhalo. Conversely, the total mass of the bound particles is of the order $M_\mathrm{tot} \gtrsim M(<r_\mathrm{max}) = \frac{r_\mathrm{max} v_\mathrm{max}^2}{G}$. 

---

## Set Parameters for the Plots

Choose the snapshot and the simulations, and define M31 and MW in each simulation. Also, set the colors used for each simulation:

In [ ]:
snap_id = 127
sim_ids = ['V1_MR_fix', 'V1_LR_fix']
names = ['plain-LCDM', 'plain-LCDM-LR']
colors = [['black', 'gray'], ['blue', 'lightblue']]
marker = ['o', '^']

m31 = [(1,0), (1,0)]
mw = [(2,0), (2,0)]

Choose how to distinguish between satellite and isolated galaxies:

In [ ]:
distinction = 'by_r'
maxdi = 2000 # Maximum distance from LG centre for isolated

Set lower limit for the value of $v_\mathrm{max}$ of the galaxies to be included (separately for satellites and isolated galaxies):

In [ ]:
sat_low = 10
isol_low = 15

---

## Retrieve Data

### Create a Dictionary

For easy handling of the relevant data, define a data dictionary that, at the top level, has entries for all simulations. Under each simulation entry, add items for the needed datasets and, under the 'Selections' key, a sub-dictionary of masking arrays for each needed condition (e.g. satellite, luminous, $v_\mathrm{max}$ inside range, etc.).

First, add the above definitions into the data dict:

In [ ]:
data = {}
for name, sim_id, m31_ns, mw_ns, col, mark in \
    zip(names, sim_ids, m31, mw, colors, marker):
    data[name] = {'snapshot': snapshot.Snapshot(sim_id, snap_id, name=name),
                  'M31_identifier': m31_ns,
                  'MW_identifier': mw_ns,
                  'PlotStyle': {
                      'Color': col,
                      'Marker': mark
                  }
                 }

Then, loop over simulations, retrieve data, compute masking arrays, and add to the dictionary:

In [ ]:
for name, sim_data in data.items():
    # Get data:
    snap = sim_data["snapshot"]
    max_point = snap.get_subhalos("Max_Vcirc", "Extended")
    vmax = max_point[:,0] * units.cm.to(units.km)
    rmax = max_point[:,1] * units.cm.to(units.kpc)
    mass = snap.get_subhalos("Mass") * units.g.to(units.Msun)
    sm = snap.get_subhalos("Stars/Mass") * units.g.to(units.Msun)
    
    # Split into satellites:
    if distinction == "by_r":
        masks_sat, mask_isol = dataset_comp.split_satellites_by_distance_old(
            snap, sim_data["M31_identifier"], sim_data["MW_identifier"])
    elif distinction == "by_gn":
        masks_sat, mask_isol = dataset_comp.split_satellites_by_group_number(
            snap, sim_data["M31_identifier"], sim_data["MW_identifier"])
        
    mask_lum, mask_dark = dataset_comp.split_luminous(snap)
    
    mask_sat_low = dataset_comp.prune_vmax(snap, low_lim=sat_low)
    mask_isol_low = dataset_comp.prune_vmax(snap, low_lim=isol_low)
    mask_m31 = masks_sat[0]
    mask_mw = masks_sat[1]
    mask_sat = np.logical_or.reduce(masks_sat)
    
    mask_pruned = np.logical_or(np.logical_and(mask_sat, mask_sat_low),
                                np.logical_and(mask_isol, mask_isol_low))

    # Add selections (masking arrays):
    data[name]['Selections'] = {
        'M31': mask_m31,
        'MW': mask_mw,
        'Satellite': mask_sat,
        'Isolated': mask_isol,
        'Luminous': mask_lum,
        'Dark': mask_dark,
        'PruneSat': mask_sat_low,
        'PruneIsol': mask_isol_low,
        'Valid': mask_pruned
    }
        
    # Add datasets to dictionary:
    data[name]['Vmax'] = vmax
    data[name]['Rmax'] = rmax
    data[name]['Mass'] = mass
    data[name]['LogSM'] = np.log10(sm)
    
    dm_mass = snap.get_particle_masses(part_type=[1])[0] \
        * units.g.to(units.Msun)
    data[name]['DM_Mass'] = dm_mass
    data[name]['Eff_DM_num'] = mass / dm_mass

---

## Plot

Plot satellites and isolated galaxies on different subplots and add median fits for each dataset.

In [ ]:
# Choose font sizes:
parameters = {'axes.titlesize': 10,
              'axes.labelsize': 9,
              'xtick.labelsize': 6,
              'ytick.labelsize': 6,
              'legend.fontsize': 8}

ms = 8 # Marker size
msl = 10
a = 0.7 # Transparency

In [ ]:
# Set marker size limits:
smin = 2; smax = 12
mmin = 10**10; mmax = 0

# Iterate through simulations to find largest value of vmax:
for sim_data in data.values():    
    mmin = min(
        mmin, np.min(sim_data['LogSM'][
            np.logical_and(sim_data['Selections']['Valid'], sim_data['LogSM'] != -np.inf)
        ])
    )    
    mmax = max(
        mmax, np.max(sim_data['LogSM'][
            np.logical_and(sim_data['Selections']['Valid'], sim_data['LogSM'] != -np.inf)
        ])
    )

def mass_to_marker(x):
    return 1/(mmax-mmin) * (smax*(x-mmin) - smin*(x-mmax))

In [ ]:
print(mmin, mmax)

In [ ]:
# Set fonts:
plt.rcParams.update(parameters)
plt.tight_layout()

fig, axes = plt.subplots(ncols=2, sharey='row', figsize=(8,4))
plt.subplots_adjust(wspace=0.05)

# Set axis:
for ax in axes:
    ax.set_xscale('log')
    ax.set_box_aspect(0.9) # Set subfigure box side aspect ratio
    
axes[0].set_xlim(2, 110)    
axes[1].set_xlim(2, 110)
axes[0].set_xlabel('$v_{\mathrm{max}}[\mathrm{km s^{-1}}]$' )
axes[1].set_xlabel('$v_{\mathrm{max}}[\mathrm{km s^{-1}}]$' )

axes[0].set_yscale('log')
# axes[0].set_ylim(15, 10**6)    
axes[0].set_ylabel('$M / M_\mathrm{DM}$' )

axes[0].set_title('Satellite Galaxies' )
axes[1].set_title('Isolated Galaxies' )

# Add scatter plots:
for i, (name, entry) in enumerate(data.items()):
    
    # Plot dark:
    mask = np.logical_and(entry['Selections']['Satellite'],
                          entry['Selections']['Dark'])
    axes[0].scatter(entry['Vmax'][mask], entry['Eff_DM_num'][mask], 
                    s=ms, edgecolor='none', alpha=a,
                    c=entry['PlotStyle']['Color'][1],
                    label="{} non-SF".format(name))
    
    mask = np.logical_and(entry['Selections']['Isolated'],
                          entry['Selections']['Dark'])
    axes[1].scatter(entry['Vmax'][mask], entry['Eff_DM_num'][mask], 
                    s=ms, edgecolor='none', alpha=a,
                    c=entry['PlotStyle']['Color'][1],
                    label="{} non-SF".format(name))
    
    # Plot luminous:
    mask = np.logical_and(entry['Selections']['Satellite'],
                          entry['Selections']['Luminous'])
    axes[0].scatter(entry['Vmax'][mask], entry['Eff_DM_num'][mask], 
                    s=mass_to_marker(entry['LogSM'][mask]),
                    facecolors='none', alpha=a,
                    edgecolors=entry['PlotStyle']['Color'][0],
                    label="{} SF".format(name))
    
    mask = np.logical_and(entry['Selections']['Isolated'],
                          entry['Selections']['Luminous'])
    axes[1].scatter(entry['Vmax'][mask], entry['Eff_DM_num'][mask], 
                    s=mass_to_marker(entry['LogSM'][mask]),
                    facecolors='none', alpha=a,
                    edgecolors=entry['PlotStyle']['Color'][0],
                    label="{} SF".format(name))

axes[1].legend(loc='upper left')

In [ ]:
text = ""

# Add scatter plots:
y_add = 0
for name, entry in data.items():
    
    # Add text box:
    axes[0].text(0.1, 0.9 - y_add, "$M_\mathrm{{DM}} = {:.2E}$\n".format(entry['DM_Mass']), 
                 transform=axes[0].transAxes, ha="left", va="top", 
                 c=entry['PlotStyle']['Color'][0])
    y_add += 0.1

fig

### Discussion

As can be seen in the figure, the minimum particle limit of SUBFIND is 20. A physically representative subhalo should, however, consist of around ~100 self-bound particles. For isolated galaxies, this is easily achieved by limiting to $v_\mathrm{max} < 15 \mathrm{km/s}$. 

Satellite galaxies of a certain size tend to have less particles than similar size isolated galaxies. This is because the lost particles are from the outer parts of the halo. These particles are tightly bound, and thus the subhalo can be considered physically meaningful even with less particles. 

Also, note that nearly all star and gas particles are less massive than the DM particles, which is why the star-forming subhalos have a different relation.

### Save the Figures

In [ ]:
filename = 'count_bound'
for name in names:
    filename += '_{}'.format(name)
filename += '.png'
    
path = os.path.abspath(os.path.join('..', 'Figures', 'MediumResolution'))
filename = os.path.join(path, filename)

fig.savefig(filename, dpi=300, bbox_inches='tight')

In [ ]:
ids = data['plain-LCDM-LR']['snapshot'].get_subhalos_IDs(part_type=[0,1,4,5])
partnums = np.array([arr.size for arr in ids])
print(np.min(partnums))